<a href="https://colab.research.google.com/github/wdittaya/MLWorkshop/blob/main/CUVIP_UnsupervisedLearningWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ทบทวนเบื้องต้น

## Clustering - การจัดกลุ่ม

เป็น unsupervised learning ที่จัดข้อมูลที่ **คล้าย** กันไว้ด้วยกัน และพยายามแยกข้อมูลที่ **ต่าง** กันออกจากกัน

โดยทั่วไปแล้ว เราจะต้องเปลี่ยนข้อมูลให้อยู่ในรูปแบบที่คำนวณความคล้ายได้ง่าย ซึ่งมักจะเป็นเวกเตอร์ของตัวเลข อย่างไรก็ตาม มาตรวัดใดๆ ที่สามารถวัดความคล้ายหรือระยะทางได้ สามารถนำมาใช้ได้ทั้งสิ้น

clustering algorithm แต่ละตัวจะ sensitive กับลักษณะของข้อมูลที่ต่างกัน เช่น k-means มักจะสร้าง cluster ที่มีลักษณะกระจุกตัวใกล้ศูนย์กลาง ในขณะที่ DBSCAN จะผูก sample ที่อยู่ใกล้กันไว้ด้วยกันโดยไม่มีรูปร่างของ cluster ที่แน่นอน การเลือกใช้ clustering algorithm ส่วนหนึ่งจึงควรพิจารณารูปแบบการกระจายตัวของข้อมูลด้วย แต่หากข้อมูลมี feature มาก (dimension มาก) การพิจารณารูปแบบการกระจายตัวของข้อมูลจะทำได้ยาก แนวทางหนึ่งอาจจะเป็นการ ลองทุก algorithm แล้วเลือกวิธีที่ได้ผลดีที่สุด

![clustering](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png)

การประเมินประสิทธิภาพของ clustering ทำได้สองรูปแบบใหญ่ๆ
1. หากมี ground truth label สามารถเปรียบเทียบข้อมูลในกลุ่ม กับ ground truth label ได้ เช่น ใช้มาตรวัดจำพวก homogeneity
2. หากไม่มี grouth truth label สามารถใช้มาตรวัดเกี่ยวกับระยะห่าง ความคล้าย ภายในและภายนอก cluster ได้ เช่น ใช้มาตรวัดจำพวก Silhouette Coefficient, DB-Index


Ref: https://scikit-learn.org/stable/modules/clustering.html


# โจทย์

1. ต้องการจัดกลุ่มยาและเวชภัณฑ์
2. ต้องการจัดกลุ่มสถานพยาบาลตามลักษณะการสั่งยาและเวชภัณฑ์

# ข้อมูลของเรา

- การสั่งยาและเวชภัณฑ์ในอังกฤษ (PDPI)
- ตารางรหัสสารเคมี (CHEM SUBS)

Ref: https://digital.nhs.uk/data-and-information/publications/statistical/practice-level-prescribing-data/december-2019

หากสนใจข้อมูลเกี่ยวกับระบบสุขภาพอังกฤษ (ก่อนปี 2022) อ่านเพิ่มเติมได้ที่ https://kb.hsri.or.th/dspace/bitstream/handle/11228/2705/p056-066.pdf

## Practice prescribing data

- SHA - Strategic Health Authority แสดงรหัสเขตงานกำกับดูแลการให้บริการ
- PCT - Primary Care Trust แสดงรหัสเขตการให้บริการสุขภาพ
- PRACTICE - รหัสสถานพยาบาล
- BNF CODE - รหัสยาและเวชภัณฑ์
- BNF NAME - ชื่อยาและเวชภัณฑ์
- ITEMS - จำนวนครั้งในการสั่งในเดือน
- NIC - ราคาตามบัญชียาหลัก
- ACT COST - ราคาที่เรียกเก็บ
- QUANTITY - ปริมาณยาและเวชภัฑณ์ เช่น 20 เม็ด
- PERIOD - ปีและเดือนของข้อมูล

## BNF code structure

This is the BNF code for the drug:

- characters 1 and 2 show the BNF chapter
- 3 and 4 show the BNF section
- 5 and 6 show the BNF paragraph
- 7 shows the BNF sub-paragraph
- 8 and 9 show the chemical substance
- 10 and 11 show the product
- 12 and13 show the strength and formulation
- 14 and 15 show the equivalent

Ref: https://digital.nhs.uk/data-and-information/areas-of-interest/prescribing/practice-level-prescribing-in-england-a-summary/practice-level-prescribing-glossary-of-terms

## Download ข้อมูล

In [ ]:
PDPI_file = 'https://files.digital.nhs.uk/82/4A18B0/T201912PDPI%20BNFT.csv'
CHEM_file = 'https://files.digital.nhs.uk/5A/F7A49D/T201912CHEM%20SUBS.csv'

In [ ]:
import pandas as pd
pdpi_df = pd.read_csv(PDPI_file, header=0)
chem_df = pd.read_csv(CHEM_file, header=0)

In [ ]:
pdpi_df.head()

In [ ]:
chem_df.head()

In [ ]:
pdpi_df.info()

In [ ]:
chem_df.info()

## Preprocessing

- ตรวจสอบและจัดการข้อมูลที่หายไป
- ตัดข้อมูลที่ไม่จำเป็น
- แปลงข้อมูลให้อยู่ในรูปแบบที่ประมวลผลได้

### ตรวจสอบข้อมูลที่หายไป

In [ ]:
pdpi_df.isna().sum()

In [ ]:
chem_df.isna().sum()

เราพบว่า คอลัมน์ `201912` ของ `chem_df` ไม่มีข้อมูล แต่เนื่องจาก header ของ column นี้ทำเพื่อบอกว่า นี่เป็นข้อมูลเดือน 12 ปี 2019 และเราใช้ข้อมูลเพียงเดือนเดียว ดังนั้น ข้อมูล column นี้ตัดทิ้งได้

In [ ]:
chem_df.drop(columns=['201912'], inplace=True)

### ตัดข้อมูลที่ไม่จำเป็น

In [ ]:
pdpi_df.describe()

ใน `pdpi_df` มี column `PERIOD` อยู่ ซึ่งมีข้อมูลเพียงค่าเดียวคือ `201912` หมายถึง ข้อมูลเดือน 12 ปี 2019

ข้อมูลนี้ไม่เป็นประโยชน์ในการวิเคราะห์ของเรา จึงตัดทิ้งทั้ง column ได้

In [ ]:
pdpi_df.drop(columns=['PERIOD'], inplace=True)

# 1. ต้องการจัดกลุ่มยาและเวชภัณฑ์

## 1.1 ใช้ BNF code structure

จาก BNF code structure เราเห็นว่ารหัส 2, 4, 6, 7, 9 หลักแรก สามารถแทนกลุ่มของยาและเวชภัณฑ์ได้ในระดับต่างๆ อยู่แล้ว

และ 9 หลักแรกนี้ คือรหัส CHEM SUB ใน `chem_df`

In [ ]:
def get_first_two_chars(bnf_code):
  if isinstance(bnf_code, str) and len(bnf_code) >= 2:
    return bnf_code[:2]
  return None

chem_df['BNF_Prefix'] = chem_df['CHEM SUB'].apply(get_first_two_chars)

prefix_counts = chem_df.groupby('BNF_Prefix')['CHEM SUB'].count()

print(prefix_counts)

In [ ]:
prefix_counts.plot(kind='bar')

เราจะลองใช้ข้อมูลนี้เป็น ground truth สำหรับประเมินการจัดกลุ่มของข้อมูล

## 1.2 ใช้ข้อความชื่อสารเคมี

In [ ]:
chem_df.head()

จากตัวอย่างจะเห็นว่า `Loperamide Hydrochloride` กับ ``Loperamide Hydrochloride & simeticone` ดูชื่อคล้ายๆ กัน ซึ่งก็น่าจะจัดกลุ่มเข้าด้วยกันได้เช่นกัน

การวัดความคล้ายของข้อความสามารถใช้ Levenshtein distance ในการวัดได้ โดยจะนับจำนวนอักขระที่ต้องแก้ไข เพื่อให้ข้อความตั้งต้นกลายเป็นข้อความปลายทาง

In [ ]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/d/d1/Levenshtein_distance_animation.gif')

ดังนั้น เราสามารถสร้าง distance matrix ของสารเคมีแต่ละคู่ได้

In [ ]:
!pip install leven

from leven import levenshtein

def levenshtein_distance_matrix(names):
  n = len(names)
  matrix = [[0] * n for _ in range(n)]
  for i in range(n):
    for j in range(i + 1, n):
      distance = levenshtein(names[i], names[j])
      matrix[i][j] = distance
      matrix[j][i] = distance
  return matrix

distance_matrix = levenshtein_distance_matrix(chem_df['NAME'].tolist())


In [ ]:
print(distance_matrix[0])

In [ ]:
# prompt: Plot distribution of pairwise distance

import matplotlib.pyplot as plt
import numpy as np

def plot_distance_distribution(distance_matrix):
  """Plots the distribution of pairwise distances from a distance matrix."""

  distances = []
  for row in distance_matrix:
    for distance in row:
      distances.append(distance)

  plt.hist(distances, bins=30)
  plt.xlabel("Pairwise Distance")
  plt.ylabel("Frequency")
  plt.title("Distribution of Pairwise Distances")
  plt.show()

# Assuming 'distance_matrix' is defined from the previous code snippet
plot_distance_distribution(distance_matrix)

เนื่องจาก DBSCAN เป็น clustering algorithm ที่ใช้ระยะห่างเป็นหลักในการตัดสินใจว่าข้อมูลจะอยู่รวมกันใน cluster หรือไม่ จึงใช้ DBSCAN ในการทดลองจัดกลุ่มสารเคมี

เราพบว่า ระยะห่างระหว่าง sample กระจายตัวอยู่ระหว่าง 5-35 ตัวอักษร แสดงว่า ข้อความที่เหมือนกันมากๆ จะมีตัวอักษรต่างกันไม่เกิน 5 ตัวอักษร

In [ ]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN(metric='precomputed', eps=8, min_samples=3).fit(distance_matrix)
chem_df['cluster'] = clustering.labels_

In [ ]:
n_clusters_ = len(set(chem_df['cluster'])) - (1 if -1 in chem_df['cluster'] else 0)
print('Estimated number of clusters: %d' % n_clusters_)


ดูจำนวนสารเคมีในแต่ละ cluster

In [ ]:
chem_df['cluster'].value_counts()

เลือก cluster ที่มีจำนวนสารเคมีค่อนข้างมาก

สำหรับ cluster -1 หมายถึง สารเคมีที่ไม่สามารถจัดกลุ่มได้ เนื่องจากไม่มีสารเคมีที่มีชื่อใกล้เคียงกันมากพอ

In [ ]:
cluster_ranking = chem_df['cluster'].value_counts().index.to_list()

In [ ]:
cluster_ranking[0]

In [ ]:
chem_df[chem_df['cluster'] == cluster_ranking[0]]['NAME'].tolist()


In [ ]:
chem_df[chem_df['cluster'] == cluster_ranking[2]]['NAME'].tolist()

In [ ]:
from sklearn.metrics import homogeneity_score
homogeneity_score(chem_df['BNF_Prefix'], chem_df['cluster'])


# 2. ให้จัดกลุ่มของสถานพยาบาล โดยใช้ข้อมูลรูปแบบการสั่งยาและเวชภัณฑ์

หารูปแบบการแทนข้อมูลที่เหมาะสม เริ่มจาก ลองดูจำนวน BNF CODE ของแต่ละสถานพยาบาลก่อน

นับจำนวนสถานพยาบาล

สร้าง k-mean cluster เลือกจำนวน cluster ที่เหมาะสม

เลือกดูข้อมูลจาก cluster ที่น่าสนใจ